<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/Model_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)
df['attribute_value'].value_counts()

casual            2633
modern            1838
classic           1488
businesscasual    1484
edgy               804
androgynous        691
romantic           558
boho               468
glam               423
athleisure         280
retro              220
Name: attribute_value, dtype: int64

In [0]:
df = pd.get_dummies(df, columns=['attribute_value'])
df = df.groupby(['product_id', 'brand', 'product_full_name', 'description', 'brand_category']).sum().reset_index()
label_list = []
for key in df.columns:
    if 'attribute_value' in key:
        df[key] = df[key].apply(lambda x: min(x, 1))
        label_list.append(key)
y = df[label_list].values
num_classes = y.shape[1]

In [0]:
X = df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']
X = X.values

In [0]:
vectorizer = TfidfVectorizer(max_features=500, min_df = 5, ngram_range = (1, 2))

In [47]:
len(vectorizer.vocabulary_)

500

In [0]:
X = vectorizer.fit_transform(X)

In [0]:
scaler = StandardScaler( with_mean = False)
X = scaler.fit_transform(X,)

In [8]:
X.reshape(-1,500)

<3344x500 sparse matrix of type '<class 'numpy.float64'>'
	with 52631 stored elements in Compressed Sparse Row format>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

def define_model():
    inputs = Input(shape = (500, ))
    embedding1_1 = Embedding(2000, 100)(inputs)
    conv1_1 = Conv1D(filters=32, kernel_size=10, activation='tanh')(embedding1_1)

    pool1_1 = MaxPooling1D(pool_size=2)(conv1_1)
    flatten = Flatten()(pool1_1)

    dense1 = Dense(64, activation = 'tanh')(flatten)
    outputs = Dense(num_classes, activation='sigmoid')(dense1)

    

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.03), metrics=['accuracy'])

    return model

In [0]:
def define_model():
    inputs = Input(shape=(500,))
    embedding = Embedding(input_dim=500, output_dim=100)(inputs)
    conv = Conv1D(filters = 32, kernel_size=10, activation='relu')(embedding)
    pool = MaxPooling1D(pool_size=2)(conv)
    flatten = Flatten()(pool)
    dense = Dense(64, activation='relu')(flatten)
    outputs = Dense(num_classes, activation='sigmoid')(dense)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

def define_model():
    model = Sequential()
    model.add(Embedding(input_dim=200, output_dim=100, input_length=500))
    model.add(Conv1D(filters=32, kernel_size=10, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    model.add(Dense(64,activation='relu'))
    model.add(Dense(11, activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [49]:
model = define_model()
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 500, 100)          50000     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 491, 32)           32032     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 245, 32)           0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 7840)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                501824    
_________________________________________________________________
dense_16 (Dense)             (None, 11)                715 

In [50]:
model.fit(X_train, y_train, validation_split=0.2, batch_size=512, epochs=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2407 samples, validate on 602 samples
Epoch 1/10
2407/2407 [==============================] - 1s 243us/step - loss: 0.6266 - accuracy: 0.7170 - val_loss: 0.5215 - val_accuracy: 0.7780
Epoch 2/10
2407/2407 [==============================] - 0s 134us/step - loss: 0.5207 - accuracy: 0.7722 - val_loss: 0.4928 - val_accuracy: 0.7765
Epoch 3/10
2407/2407 [==============================] - 0s 128us/step - loss: 0.4871 - accuracy: 0.7807 - val_loss: 0.4897 - val_accuracy: 0.7780
Epoch 4/10
2407/2407 [==============================] - 0s 127us/step - loss: 0.4835 - accuracy: 0.7823 - val_loss: 0.4782 - val_accuracy: 0.7765
Epoch 5/10
2407/2407 [==============================] - 0s 127us/step - loss: 0.4791 - accuracy: 0.7787 - val_loss: 0.4767 - val_accuracy: 0.7801
Epoch 6/10
2407/2407 [==============================] - 0s 131us/step - loss: 0.4777 - accuracy: 0.7807 - val_loss: 0.4755 - val_accuracy: 0.7780
Epoch 7/10
2407/2407 [==============================] - 0s 128us/step - loss:

In [51]:
model.evaluate(X_test, y_test)

335/335 [==============================] - 0s 124us/step


[0.46973077236716426, 0.781818151473999]

In [52]:
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()

4    3189
8     155
dtype: int64